# 🏆 Proyecto Integrador Senior 1: Plataforma de Datos Completa

Objetivo: diseñar e implementar una plataforma moderna de datos con governance, lakehouse, orquestación, observabilidad y compliance.

- Duración: 180+ min (proyecto multi-día)
- Dificultad: Muy Alta
- Prerrequisitos: Todos los notebooks Senior 01–08

## 1. Contexto y requerimientos

**Empresa**: E-commerce global con 3 dominios de datos (Ventas, Logística, Analítica).

**Requerimientos funcionales**:
- Ingestar transacciones en tiempo real (Kafka) y batch nocturno (archivos SFTP).
- Almacenar en data lakehouse (Parquet + Delta Lake) particionado por fecha y región.
- Catálogo central con metadatos, linaje y políticas de acceso (Glue/Unity/DataHub).
- Orquestación diaria con Airflow: validaciones de calidad, transformaciones, reportes.
- APIs de servicio (FastAPI) para consultas ad-hoc por BI y científicos de datos.

**Requerimientos no funcionales**:
- Compliance GDPR: enmascaramiento de PII, derecho al olvido.
- SLO: latencia p99 < 30 min, disponibilidad > 99.5%.
- Costos: < $5000/mes, optimización continua (FinOps).
- Observabilidad: logs estructurados, métricas en Prometheus, linaje en DataHub.
- Seguridad: IAM con mínimo privilegio, cifrado at-rest y in-transit, auditoría.

## 2. Arquitectura propuesta

In [ ]:
arquitectura_diagrama = '''
┌─────────────┐       ┌──────────────┐       ┌───────────────┐
│  Transac-   │──────▶│    Kafka     │──────▶│  Spark        │
│  ciones RT  │       │  (streaming) │       │  Streaming    │
└─────────────┘       └──────────────┘       └───────┬───────┘
                                                      │
┌─────────────┐       ┌──────────────┐              │
│  Archivos   │──────▶│   Airflow    │──────────────┤
│  SFTP Batch │       │  (orquesta)  │              │
└─────────────┘       └──────────────┘              │
                                                     ▼
                      ┌──────────────────────────────────┐
                      │  Data Lakehouse (S3 + Delta)     │
                      │  - raw/                          │
                      │  - curated/                      │
                      │  - gold/ (agregados)             │
                      └─────────┬────────────────────────┘
                                │
                ┌───────────────┼───────────────┐
                ▼               ▼               ▼
         ┌──────────┐   ┌──────────┐   ┌──────────┐
         │  Athena  │   │ FastAPI  │   │   BI     │
         │  (SQL)   │   │ (APIs)   │   │ (Tableau)│
         └──────────┘   └──────────┘   └──────────┘

Observabilidad: Prometheus + Grafana + DataHub (linaje)
Seguridad: IAM, KMS, CloudTrail, enmascaramiento PII
'''
print(arquitectura_diagrama)

## 3. Componentes a implementar (checklist)

In [ ]:
checklist = '''
☐ 1. Kafka cluster (Docker Compose local o MSK en AWS)
☐ 2. Productor de eventos simulados (transacciones)
☐ 3. Consumidor Spark Streaming → Delta Lake (S3)
☐ 4. Airflow DAG batch: SFTP → raw → validación → curated → gold
☐ 5. Validaciones de calidad con Great Expectations
☐ 6. Enmascaramiento de PII (email, tarjeta)
☐ 7. Catálogo con Glue Data Catalog o DataHub
☐ 8. Linaje con OpenLineage (plugin Airflow)
☐ 9. FastAPI endpoint para consultas SQL (proxy a Athena/Trino)
☐ 10. Métricas Prometheus exportadas por pipelines
☐ 11. Dashboard Grafana con SLOs y alertas
☐ 12. Políticas IAM con mínimo privilegio
☐ 13. Cifrado KMS para S3 y RDS
☐ 14. Auditoría CloudTrail habilitada
☐ 15. Presupuestos y alertas de costos (AWS Budgets)
☐ 16. Documentación técnica y runbooks
☐ 17. Tests de integración (Pytest)
☐ 18. CI/CD con GitHub Actions (lint, test, deploy)
'''
print(checklist)

## 4. Implementación paso a paso

### 4.1 Setup inicial
- Crear bucket S3 con estructura `raw/`, `curated/`, `gold/`.
- Configurar Glue Data Catalog con base de datos `ecommerce`.
- Levantar Kafka local con Docker Compose (zookeeper + broker).

### 4.2 Streaming path
- Productor Python: genera eventos JSON (transacción_id, cliente_id, monto, timestamp).
- Spark Structured Streaming: consume de Kafka, valida schema, escribe a Delta en `curated/ventas/`.
- Checkpointing idempotente.

### 4.3 Batch path
- Airflow DAG: sensor SFTP → download → validate (GE) → transform (Pandas/Spark) → write Delta → optimize.
- Agregaciones gold: ventas por día/región/producto.

### 4.4 Governance y seguridad
- Enmascarar email y tarjeta antes de escribir en curated.
- Registrar linaje en DataHub vía OpenLineage.
- Configurar IAM roles para Spark, Airflow, APIs.

### 4.5 Observabilidad
- Exportar métricas de conteo, latencia, errores.
- Dashboard Grafana con paneles por pipeline.
- Alertas en Slack si SLO violado.

### 4.6 Servicio de consultas
- FastAPI endpoint `/query` que ejecuta SQL en Athena y retorna JSON.
- Caché Redis para queries repetitivas.
- Rate limiting por API key.

## 5. Entregables

- Repositorio Git con código (pipelines, DAGs, APIs, tests).
- Diagrama de arquitectura actualizado.
- Documento de diseño (decisiones técnicas, trade-offs).
- Dashboard Grafana exportado (JSON).
- Runbook de operaciones (troubleshooting, rollback).
- Video/demo ejecutando pipeline end-to-end.

## 6. Evaluación

- Funcionalidad: ¿pipelines ejecutan correctamente?
- Calidad: ¿validaciones y tests implementados?
- Observabilidad: ¿métricas y linaje visibles?
- Seguridad: ¿IAM, cifrado, PII enmascarado?
- Costos: ¿presupuesto respetado?
- Documentación: ¿clara y completa?